## Extracción y selección de datos

In [1]:
import pandas as pd

xls = pd.ExcelFile('delitos.xlsx')
sheet_names = xls.sheet_names
# Diccionario para almacenar los nacionalFrames por estado
data_por_estado = {}
# Cargar los nacionalFrames para cada estado
for estado in sheet_names:
    # Cargar la hoja correspondiente a cada estado
    data_por_estado[estado] = pd.read_excel('delitos.xlsx', sheet_name=estado)

In [2]:
# Selección de nombres de fila relevantes en la columna "Tipo de delito, subtipo y modalidad"
delitos = ["Extorsión","Homicidio",
           "Feminicidio","Secuestro","Violación simple",
           "Abuso sexual","Hostigamiento sexual",
           "Violencia familiar","Robo te vehículo automotor",
           "Homicidio doloso","Robo a casa habitación","Robo a transportista",
           "Robo a transeúnte en vía pública",
           "Robo en transporte público colectivo","Robo de maquinaria",
           "Violencia de género en todas sus modalidades distinta a la violencia familiar"]

In [3]:
# función de limpieza de la tabla de datos
def procesar_data_est(df):
    df["Unnamed: 3"] = df["Unnamed: 3"].fillna(df["Unnamed: 4"])
    df["Unnamed: 3"] = df["Unnamed: 3"].fillna(df["SECRETARIADO EJECUTIVO DEL SISTEMA NACIONAL DE SEGURIDAD PÚBLICA"])
    
    try:
        df.drop(['Unnamed: 0',"SECRETARIADO EJECUTIVO DEL SISTEMA NACIONAL DE SEGURIDAD PÚBLICA",
                 "Unnamed: 4","Unnamed: 5","Unnamed: 6"], axis=1, inplace=True, errors = "ignore")
    except KeyError:
        pass
    df.rename(columns={"Unnamed: 2":"Clave",
                       "Unnamed: 3":"Tipo de delito subtipo y modalidad",
                       "Unnamed: 7":"Enero",
                       "Unnamed: 8":"Febrero",
                       "Unnamed: 9":"Marzo",
                       "Unnamed: 10":"Abril",
                       "Unnamed: 11":"Mayo",
                       "Unnamed: 12":"Junio",
                       "Unnamed: 13":"Julio",
                       "Unnamed: 14":"Agosto",
                       "Unnamed: 15":"Septiembre",
                       "Unnamed: 16":"Octubre",
                       "Unnamed: 17":"Noviembre",
                       "Unnamed: 18":"Diciembre",
                       "Unnamed: 19":"Total"}, inplace=True)
    
    return df

In [5]:
# 'dataframes_dict' es un diccionario que contiene DataFrames
dataframes_procesados = {}
for key, df in data_por_estado.items():
    df_procesado = procesar_data_est(df)
    filtro = df_procesado['Tipo de delito subtipo y modalidad'].isin(delitos)
    df_filtrado = df_procesado[filtro]
    nombre_estado = key.replace(". ", "_").lower()
    dataframes_procesados[nombre_estado] = df_filtrado

In [9]:
dataframes_procesados['15_méx']

,Clave,Tipo de delito subtipo y modalidad,Enero,Febrero,Marzo,Abril,Mayo,Junio,Julio,Agosto,Septiembre,Octubre,Noviembre,Diciembre,Total
9,1.1,Homicidio,328,315,309,335,300,323,315,283,329,298,NaN,NaN,3135
10,1.1.1,Homicidio doloso,216,186,181,205,179,192,201,182,214,169,NaN,NaN,1925
33,1.3,Feminicidio,7,11,7,11,6,9,4,9,8,8,NaN,NaN,80
41,2.1,Secuestro,6,7,6,6,11,2,5,4,6,3,NaN,NaN,56
51,3.1,Abuso sexual,393,456,563,462,470,582,428,400,441,453,NaN,NaN,4648
53,3.3,Hostigamiento sexual,27,17,21,9,12,30,62,15,28,21,NaN,NaN,242
54,3.4,Violación simple,163,180,225,162,229,239,149,220,253,260,NaN,NaN,2080
60,4.1.1,Robo a casa habitación,772,708,762,732,680,781,833,940,766,776,NaN,NaN,7750
76,4.1.4,Robo a transportista,394,322,381,305,347,357,320,370,367,357,NaN,NaN,3520
79,4.1.5,Robo a transeúnte en vía pública,1767,1837,1923,1640,1550,1766,1468,1879,1778,1687,NaN,NaN,17295


In [10]:
# Se guarda en un archivo de tipo xlsx que contiene 32 páginas, una por cada estado
with pd.ExcelWriter('IDFC_2023.xlsx') as writer:
    # Iterar sobre los DataFrames procesados y filtrados
    for nom_est, df in dataframes_procesados.items():
        # Guardar cada DataFrame en una hoja del archivo Excel
        df.to_excel(writer, sheet_name=nom_est, index=False)